## Team members
Username1： kl2912@columbia.edu,  Registered email address: kl2912@columbia.edu

Username2:  zs2319 ,              Registered email address: zs2319@columbia.edu



1. Describe your entity resolution technique, as well as its precision, recall, and F1 score.

First, we did data processing and feature selection over the two data sets amazon.csv and rotten_tomatoes.csv. Since the same features that two data set shared were "time", "director" and "star", we decided to choose the three features to make up our classifier, thus dropped the unrelated columns like "cost" in amazon.csv, and audience rating and review relevant columns in rotten_tomatoes.csv.
After that, we found out the two datasets did not have the same exact formatting for time and star. Therefore, we normalized the format of time features by using regular expression operations. In terms of the standardization of the star name features, we managed to build a set to contain all the star names appearing in each row to prepare for the 
intersection operation in the next step. 
After cleansing the two data sets, we created a data transformation function which basically would apply to train and test dataset. It added three new columns to the original data set, that are "time_diff", "director_diff" and "star_diff". "time_diff" was the absolute difference of the times, while "director_diff" was calculated by the string distance operation. Last but not least, "director_diff" was the length of intersection of two star name sets of id1 in amazon.csv and id2 in rotten_tomatoes.csv respectively.
Then we applied the mean inputer and random forest classifier to fit our proccessed train data set to get our classifier. The precision, recall, and F1 score we got are all 1.0.
At this point, the scores of train set was high enough, so we applied the classifier to predict the test dataset and got f1-score 96.25%.

2. What were the most important features that powered your technique?
There were three most importance features, which are "time", "director" and "star".

3. How did you avoid pairwise comparison of all movies across both datasets?
We created a data transformation function which basically would apply to train and test dataset. 
It added three new columns ("time_diff", "director_diff" and "star_diff") to the original data set, of which "time_diff" and "director_diff" stood for the absolute time difference and the normalized director string distance respectively between the row with id1 in amazon.csv and the row with id2 in rotten_tomatoes.csv, while "star_diff" represented the number of overlapped star names between the two rows.
Through this step, we avoided pairwise comparison of all movies across both datasets.

In [1]:
import numpy as np

import pandas as pd

import re as re

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import Imputer

# read data

amazon = pd.read_csv("amazon.csv")

rotten_tomatoes = pd.read_csv("rotten_tomatoes.csv", encoding="ISO-8859-1")

train = pd.read_csv("train.csv")

test = pd.read_csv("test.csv")

holdout = pd.read_csv("holdout.csv")

rotten_selected = rotten_tomatoes.iloc[:, :10]

## amazon data processing

# deal with time period

amazon = amazon.drop('cost', 1)

for i in range(amazon.shape[0]):

    starcol = str(amazon.loc[i]['star'])

    if bool(re.search(r'\d', starcol)):
        amazon.set_value(i, 'time', starcol)

        amazon.set_value(i, 'star', np.nan)  # cope with time column

    timelong = str(amazon.loc[i]['time'])

    if timelong.find('/') != -1:

        amazon.set_value(i, 'time', np.nan)

    else:

        h = int(timelong.find('hour'))

        mins = int(timelong.find('minute'))

        comma = int(timelong.find(','))

        if h != -1:

            hours = int(timelong[h - 2])

            minutes = int(timelong[comma + 2:mins - 1])

        elif mins != -1:

            hours = 0

            minutes = int(timelong[:mins - 1])

        else:

            amazon.set_value(i, 'time', np.nan)

        amazon.set_value(i, 'time', hours * 60 + minutes)

# deal with star names

for i in range(amazon.shape[0]):

    # deal with star

    starcol = str(amazon.loc[i]['star'])

    if starcol.find('/') == -1:

        starcol = set(starcol.split(', '))

    else:

        starcol = set(starcol.split('/'))

    amazon.set_value(i, 'star', starcol)

amazon_selected = amazon
## rotten tomatoes data cleansing

# deal with time

prob = []

for index, row in rotten_selected.iterrows():

    timelong = row['time']

    if str(timelong) != 'nan':

        m = re.search(r'(\d+) hr. (\d+) min.', timelong)

        m1 = re.search(r'(\d+) hr.', timelong)

        m2 = re.search(r'(\d+) min.', timelong)

        if m:

            time_tup = m.groups()

            minute = int(time_tup[0]) * 60 + int(time_tup[1])

        elif m1:

            time_tup = m1.groups()

            minute = int(time_tup[0]) * 60

        elif m2:

            time_tup = m2.groups()

            minute = int(time_tup[0])

        else:

            prob.append(index)

        rotten_selected.set_value(index, 'time', minute)

rotten_selected = rotten_selected.drop('year', 1)

# combine star1 to star6 into a column of sets star

rotten_selected['star'] = [set() for x in range(len(rotten_selected.index))]

for index, row in rotten_selected.iterrows():

    star = set()

    for i in range(3, 9):

        if str(row[i]) != 'nan':
            star.add(row[i])

    rotten_selected.set_value(index, 'star', star)

rotten_selected = rotten_selected.iloc[:, [0, 1, 2, 9]]

# transform training set

import distance


def transformation(X):
    """

    transform train into standard format

    """

    X['time_diff'] = 0

    X['director_diff'] = 0

    X['star_diff'] = 0

    for i in range(X.shape[0]):
        id_amazon = X.iloc[i, 0]

        id_rotten = X.iloc[i, 1]

        row_amazon = amazon_selected[amazon_selected['id'] == id_amazon]

        row_rotten = rotten_selected[rotten_selected['id'] == id_rotten]

        X.iloc[i, 3] = np.absolute(float(row_amazon['time']) - float(row_rotten['time']))

        X.iloc[i, 4] = distance.levenshtein(str(row_amazon['director']), str(row_rotten['director']), normalized=True)

        # the intersection of star names

        amazon_star = list(row_amazon['star'])[0]

        rotten_star = list(row_rotten['star'])[0]

        X.iloc[i, 5] = len(amazon_star.intersection(rotten_star))

    X_train = X.iloc[:, [0, 1, 3, 4, 5]]

    y_train = X.iloc[:, 2]

    return X_train, y_train

X_train,y_train=transformation(train)

test['gold'] = 0
X_test, y_test = transformation(test)
holdout['gold'] = 0
X_holdout, y_holdout = transformation(holdout)


imp = Imputer(strategy="mean").fit(X_train)
X_train_imp = imp.transform(X_train)
X_test_imp = imp.transform(X_test)
X_holdout_imp = imp.transform(X_holdout)

# rf = RandomForestClassifier(n_estimators=15)
# rf.fit(X_train_imp[:,2:5],y_train)
# rf.score(X_train_imp[:,2:5], y_train)

from sklearn.ensemble import AdaBoostClassifier
#rf = AdaBoostClassifier(base_estimator=RandomForestClassifier, n_estimators=20)

rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_train_imp[:,2:5],y_train)

rf.score(X_train_imp[:,2:5], y_train)

pred = rf.predict(X_test_imp[:,2:5])

from sklearn.metrics import f1_score,precision_score,recall_score

train_pred=rf.predict(X_train_imp[:,2:5])

print(rf.score(X_train_imp[:,2:5], y_train))

print(f1_score(y_train,train_pred,average="macro"))

print(precision_score(y_train,train_pred,average="macro"))

print(recall_score(y_train,train_pred,average="macro"))

prediction = pd.DataFrame(pred,columns=['gold'])

prediction = prediction.to_csv('gold.csv',index=False)

prediction2 = pd.DataFrame(rf.predict(X_holdout_imp[:, 2:5]), columns=['gold'])

prediction2 = prediction2.to_csv('gold2.csv',index=False)


1.0
1.0
1.0
1.0
